## Scraping
We scrape our data from [Zillow](https://www.zillow.com/) which is a real estate website. We are going to scrape the data of sold houses in Los Angeles, California. We divide our scraping into two parts:

* First we scrape the links of the houses that are sold in Los Angeles, California in the past 90 days.
* Then we scrape the data of each house from the links we got in the first part.

### Part 1: Scraping the links of the houses

We use `selenium` to scrape the links of the houses. There was a problem in scraping the links of the houses because the website gives only the first 20 pages of our query. To solve this problem, we divided the houses by price range and scraped the links of the houses in each price range separately. `search_links` is the link of the first page of the houses that are sold in Los Angeles, California in the past 90 days in a specific price range.  `scrape_house_links` gets the link of the first page of the search query and scrape all of the house links in the search query.

In [32]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def scrape_house_links(starting_page_link, file_name, max_pages = 20):

    file = open(f"house-links/{file_name}.txt", "w")

    options = webdriver.ChromeOptions() 
    options.add_argument("--disable-blink-features=AutomationControlled") 
    options.add_experimental_option("excludeSwitches", ["enable-automation"]) 
    options.add_experimental_option("useAutomationExtension", False) 
    
    driver = webdriver.Chrome(options=options) 
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})") 
    driver.set_window_size(800, 600)
    
    driver.get(starting_page_link)

    for i in range(max_pages):
        print(i)
        element_present = EC.presence_of_element_located((By.ID, 'grid-search-results'))
        WebDriverWait(driver, timeout=10).until(element_present)
        time.sleep(1)
        total_height = driver.execute_script("return document.body.scrollHeight")

        scroll_position = 0

        while scroll_position < total_height:
            scroll_position += 500
            driver.execute_script("window.scrollTo(0, {});".format(scroll_position))
            time.sleep(0.5)
        
        
        link_elements = driver.find_elements(By.CLASS_NAME, "property-card-link")
        links = {link.get_attribute("href") for link in link_elements}

        for link in links:
            file.write(link + "\n")
            
        next_button = driver.find_element(By.XPATH, '//*[@id="grid-search-results"]/div[2]/nav/ul/li[10]/a')
        driver.execute_script("window.scrollTo(0, arguments[0].scrollHeight)", next_button)

        try:
            next_button.click()
        except:
            break
            

    file.close()

In [27]:
search_links = [
    "https://www.zillow.com/los-angeles-ca/sold/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22west%22%3A-118.95143586914062%2C%22east%22%3A-117.87202913085937%2C%22south%22%3A33.67720212849842%2C%22north%22%3A34.36355099753262%7D%2C%22usersSearchTerm%22%3A%22Los%20Angeles%2C%20CA%22%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A12447%2C%22regionType%22%3A6%7D%5D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22pricea%22%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22rs%22%3A%7B%22value%22%3Atrue%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22doz%22%3A%7B%22value%22%3A%2290%22%7D%2C%22price%22%3A%7B%22min%22%3A1%2C%22max%22%3A550000%7D%2C%22mp%22%3A%7B%22min%22%3A0%2C%22max%22%3A2878%7D%7D%2C%22isListVisible%22%3Atrue%7D",
    "https://www.zillow.com/los-angeles-ca/sold/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22west%22%3A-118.95143586914062%2C%22east%22%3A-117.87202913085937%2C%22south%22%3A33.67720212849842%2C%22north%22%3A34.36355099753262%7D%2C%22usersSearchTerm%22%3A%22Los%20Angeles%2C%20CA%22%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A12447%2C%22regionType%22%3A6%7D%5D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22pricea%22%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22rs%22%3A%7B%22value%22%3Atrue%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22doz%22%3A%7B%22value%22%3A%2290%22%7D%2C%22price%22%3A%7B%22min%22%3A550001%2C%22max%22%3A720000%7D%2C%22mp%22%3A%7B%22min%22%3A2878%2C%22max%22%3A3663%7D%7D%2C%22isListVisible%22%3Atrue%7D",
    "https://www.zillow.com/los-angeles-ca/sold/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22west%22%3A-118.95143586914062%2C%22east%22%3A-117.87202913085937%2C%22south%22%3A33.67720212849842%2C%22north%22%3A34.36355099753262%7D%2C%22usersSearchTerm%22%3A%22Los%20Angeles%2C%20CA%22%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A12447%2C%22regionType%22%3A6%7D%5D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22pricea%22%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22rs%22%3A%7B%22value%22%3Atrue%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22doz%22%3A%7B%22value%22%3A%2290%22%7D%2C%22price%22%3A%7B%22min%22%3A720001%2C%22max%22%3A870000%7D%2C%22mp%22%3A%7B%22min%22%3A3767%2C%22max%22%3A4448%7D%7D%2C%22isListVisible%22%3Atrue%7D",
    "https://www.zillow.com/los-angeles-ca/sold/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22west%22%3A-118.95143586914062%2C%22east%22%3A-117.87202913085937%2C%22south%22%3A33.67720212849842%2C%22north%22%3A34.36355099753262%7D%2C%22usersSearchTerm%22%3A%22Los%20Angeles%2C%20CA%22%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A12447%2C%22regionType%22%3A6%7D%5D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22pricea%22%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22rs%22%3A%7B%22value%22%3Atrue%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22doz%22%3A%7B%22value%22%3A%2290%22%7D%2C%22price%22%3A%7B%22min%22%3A870001%2C%22max%22%3A1070000%7D%2C%22mp%22%3A%7B%22min%22%3A3767%2C%22max%22%3A6541%7D%7D%2C%22isListVisible%22%3Atrue%7D",
    "https://www.zillow.com/los-angeles-ca/sold/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22west%22%3A-118.95143586914062%2C%22east%22%3A-117.87202913085937%2C%22south%22%3A33.67720212849842%2C%22north%22%3A34.36355099753262%7D%2C%22usersSearchTerm%22%3A%22Los%20Angeles%2C%20CA%22%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A12447%2C%22regionType%22%3A6%7D%5D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22pricea%22%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22rs%22%3A%7B%22value%22%3Atrue%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22doz%22%3A%7B%22value%22%3A%2290%22%7D%2C%22price%22%3A%7B%22min%22%3A1070001%2C%22max%22%3A1350000%7D%2C%22mp%22%3A%7B%22min%22%3A5599%2C%22max%22%3A6541%7D%7D%2C%22isListVisible%22%3Atrue%7D",
    "https://www.zillow.com/los-angeles-ca/sold/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22west%22%3A-118.95143586914062%2C%22east%22%3A-117.87202913085937%2C%22south%22%3A33.67720212849842%2C%22north%22%3A34.36355099753262%7D%2C%22usersSearchTerm%22%3A%22Los%20Angeles%2C%20CA%22%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A12447%2C%22regionType%22%3A6%7D%5D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22pricea%22%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22rs%22%3A%7B%22value%22%3Atrue%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22doz%22%3A%7B%22value%22%3A%2290%22%7D%2C%22price%22%3A%7B%22min%22%3A1350001%2C%22max%22%3A1850000%7D%2C%22mp%22%3A%7B%22min%22%3A7064%2C%22max%22%3A9157%7D%7D%2C%22isListVisible%22%3Atrue%7D",
    "https://www.zillow.com/los-angeles-ca/sold/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22west%22%3A-118.95143586914062%2C%22east%22%3A-117.87202913085937%2C%22south%22%3A33.67720212849842%2C%22north%22%3A34.36355099753262%7D%2C%22usersSearchTerm%22%3A%22Los%20Angeles%2C%20CA%22%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A12447%2C%22regionType%22%3A6%7D%5D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22pricea%22%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22rs%22%3A%7B%22value%22%3Atrue%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22doz%22%3A%7B%22value%22%3A%2290%22%7D%2C%22price%22%3A%7B%22min%22%3A1850001%2C%22max%22%3A3500000%7D%2C%22mp%22%3A%7B%22min%22%3A7064%2C%22max%22%3A18314%7D%7D%2C%22isListVisible%22%3Atrue%7D",
    "https://www.zillow.com/los-angeles-ca/sold/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22west%22%3A-118.95143586914062%2C%22east%22%3A-117.87202913085937%2C%22south%22%3A33.67720212849842%2C%22north%22%3A34.36355099753262%7D%2C%22usersSearchTerm%22%3A%22Los%20Angeles%2C%20CA%22%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A12447%2C%22regionType%22%3A6%7D%5D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22pricea%22%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22rs%22%3A%7B%22value%22%3Atrue%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22doz%22%3A%7B%22value%22%3A%2290%22%7D%2C%22price%22%3A%7B%22min%22%3A3500001%7D%2C%22mp%22%3A%7B%22min%22%3A18314%7D%7D%2C%22isListVisible%22%3Atrue%7D"
]

Now for each search link we call `scrape_house_links` to scrape the links of houses.

In [ ]:
for i, starting_link in enumerate(search_links[]):
    print(f"reading link:{i}")
    try:
        scrape_house_links(starting_link, f"houses{i}")
    except:
        print(f"exception happened at iteration{i}")
        break

### Part 2: Scraping the details of each house

In this part we use `requests` library to get the html of each house's page and then by utilizing `BeautifulSoup` library we find the data of the house with `#__NEXT_DATA__` element id. we convert this data to json and after that we save the specific features that we want to have in our dataset. we also write a function to convert this data to a row in csv file.
We had a problem that the site would give us 403 error after some requests. To solve this problem we sent the request with a header that makes the site think we are human and we also resend the request until we get the response.

In [40]:
from requests import get
from requests.exceptions import HTTPError, ConnectionError 
from bs4 import BeautifulSoup
from json import loads
from html import unescape
from re import compile
from fake_useragent import UserAgent


FEATURES = ["price", "streetAddress", "zipcode", "bedrooms", "bathrooms", "latitude",
                "longitude", "homeStatus", "homeType", "lotAreaValue", "lotAreaUnits",
                "zestimate", "rentZestimate", "currency", "yearBuilt", "livingAreaValue",
                "livingAreaUnitsShort", "taxAssessedValue"]
    
RESOFACTS = ["bathroomsFull", "pricePerSquareFoot", "stories", "heating", "cooling", "flooring", "fireplaceFeatures", "foundationDetails",
            "garageParkingCapacity", "sewer", "roofType"]

COLUMNS = ["zipcode", "streetAddress", "homeStatus", "homeType", "latitude", "longitude", "yearBuilt",
           "lotAreaValue", "lotAreaUnits", "livingAreaValue", "livingAreaUnitsShort", "garageParkingCapacity",
           "bedrooms", "bathrooms", "bathroomsFull", "stories", "flooring", "foundationDetails", "heating",
           "cooling", "fireplaceFeatures", "sewer", "roofType", "taxAssessedValue", "zestimate", "rentZestimate", "sold-history",
           "pricePerSquareFoot", "price", "currency"]

def get_nested_value(dic, key_path):
    keys = key_path.split(".")
    current = dic
    for key in keys:
        current = current.get(key, {})
        if current == {} or current is None:
            return None
    return current

def remove_space(value):
    regex_space = compile(r"[\s ]+")
    return regex_space.sub(" ", value.strip())

ua = UserAgent()

def get_house_data(house_link):
    user_agent = next(user_agent_cycle)
    
    headers = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "Accept-Language": "en-US,en;q=0.9",
        "Cache-Control": "no-cache",
        "Pragma": "no-cache",
        "Sec-Ch-Ua": '"Chromium";v="120", "Google Chrome";v="120", "Not;A=Brand";v="8"',
        "Sec-Ch-Ua-Mobile": "?0",
        "Sec-Ch-Ua-Platform": '"Windows"',
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "same-origin",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": ua.random,
        "Referer": "https://www.google.com/",
        "Connection": "keep-alive",
        "DNT": "1",  
    }   
    try:
        response = get(url=house_link, headers=headers)
        response.raise_for_status()
    except HTTPError as e:
        return get_house_data(house_link)
    except ConnectionError as e:
        return get_house_data(house_link)
    soup = BeautifulSoup(response.content, "html.parser")
    selections = soup.select("#__NEXT_DATA__")
    if selections:
        htmlData = soup.select("#__NEXT_DATA__")[0].getText()
        htmlData = remove_space(unescape(htmlData))
        data = loads(htmlData)
        property_json = get_nested_value(data,"props.pageProps.componentProps.gdpClientCache")
        property_json = loads(property_json)
        for data in property_json.values():
            if "property" in str(data) :
                parsed_data = data.get("property")
    return parsed_data


def fetch_house_features(house_data):
    feature_values = {} 
    
    for feature in FEATURES:
        feature_values[feature] = house_data.get(feature)
        
    for resofact in RESOFACTS:
        feature_values[resofact] = house_data.get("resoFacts", {}).get(resofact)
        
    sold_values = []
    price_history = house_data.get('priceHistory')
    for ph in price_history:
        if ph.get('event') == 'Sold':
            sold_values.append((ph.get('date'), ph.get('price')))
    feature_values["sold-history"] = sold_values[1:]
    
    return feature_values

def scrape_house_data(house_link):
    house_data = get_house_data(house_link)
    return fetch_house_features(house_data)

def convert_feature_values_to_csv_row(feature_values):
    row = ""
    for col in COLUMNS:
        cell_value = feature_values.get(col, '')
        if type(cell_value) == list:
            cell_value = f'"{cell_value}"'
        else:
            cell_value = str(cell_value)
            
        row += f"{cell_value},"
    return row
        

Now we write a function(`scrape_houses`) that scrape all of the links that we had found in the first part.

In [7]:
import tqdm
import csv
import time

def scrape_houses(index, start_index = 0, end_index = -1):
    input_file = open(f"houses{index}.txt", "r")
    links = input_file.readlines()
    input_file.close()
    output_file = open(f"houses.csv", "a+")
    if end_index == -1:
        end_index = len(links)
    for i in tqdm.tqdm(range(start_index, end_index)):
        try:
            house_data = scrape_house_data(links[i])
            row = convert_feature_values_to_csv_row(house_data)
            output_file.write(row + "\n")
        except Exception as e:
            print(f"error at index {i}")
            print(e)
            output_file.close()
            break
    output_file.close()

We also write a function to setup the csv file by writing the header of the csv file.

In [6]:
def setup_csv_file():
    file = open(f"houses.csv", "w")
    for col in COLUMNS:
        file.write(f"{col},")
    file.write("\n")
    file.close()

Now we call `scrape_houses` for each file that we had found in the first part to scrape the data of the houses.

In [ ]:
for i in range(8):
    scrape_houses(i)

We had a bug in our code that we could have comma in the data of one of the columns which would make the csv file to have more columns than we want. To solve this problem we put " " around the data of that specific column.

In [ ]:
import csv

input_file = 'houses.csv'
output_file = 'modified_data.csv'

def is_number(s):
    if s == 'None':
        return True
    try:
        float(s)
        return True
    except ValueError:
        return False

def process_row(row):
    while len(row) > 23 and not is_number(row[23]):
        row[22] += ',' + row[23]
        del row[23]
    return row

with open(input_file, 'r', newline='') as infile, open(output_file, 'w', newline='') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)
    
    header = next(reader)  
    writer.writerow(header)
    
    for row in reader:
        if len(row) > 23:
            row = process_row(row)
        writer.writerow(row)


After the scraping the data we found out that we needed some more features to have a better dataset. We added the features that we needed to the code and scraped the remaining features like we did before.

In [ ]:
ADDITIONAL_FEATURES = ['parkingCapacity', 'hasCooling', 'hasHeating', 'hasFireplace', 'hasPrivatePool', 'hasSpa',
                           'hasView', 'securityFeatures', ]

def fetch_additional_features(house_data):
    feature_values = {}
            
    for resofact in ADDITIONAL_FEATURES:
        feature_values[resofact] = house_data.get("resoFacts", {}).get(resofact)
        
    
    return feature_values

def convert_additional_features_to_csv(feature_values):
    row = ""
    for col in ADDITIONAL_FEATURES:
        cell_value = feature_values.get(col, '')
        if type(cell_value) == list:
            cell_value = f'"{cell_value}"'
        else:
            cell_value = str(cell_value)
            
        row += f"{cell_value},"
    return row

def scrape_additional_features_data(house_link):
    house_data = get_house_data(house_link)
    return fetch_additional_features(house_data)

def scrape_additional_features(index, start_index = 0, end_index = -1):
    input_file = open(f"houses{index}.txt", "r")
    prev_file = open("modified_data.csv", "r")
    data_prev = prev_file.readlines()
    row_nums = [765, 759, 794, 795, 770, 711, 729, 283]
    links = input_file.readlines()
    input_file.close()
    output_file = open(f"modfied_data2.csv", "a+")
    if end_index == -1:
        end_index = len(links)
    for i in tqdm.tqdm(range(start_index, end_index)):
        try:
            house_data = scrape_additional_features_data(links[i])
            row = convert_additional_features_to_csv(house_data)
            row_num = sum(row_nums[:index]) + i + 1
            row_before = data_prev[row_num]
            output_file.write(row_before[:-1] + row + "\n")
        except Exception as e:
            print(f"error at index {i}")
            print(e)
            output_file.close()
            break
    output_file.close()
    
def setup_additional_csv_file():
    file = open(f"modfied_data2.csv", "w")
    for col in COLUMNS:
        file.write(f"{col},")
    for col in ADDITIONAL_FEATURES:
        file.write(f"{col},")
    file.write("\n")
    file.close()


In [41]:
for i in range(1, 8):
    scrape_additional_features(i)

  0%|          | 0/759 [00:00<?, ?it/s]

100%|██████████| 283/283 [21:43<00:00,  4.61s/it]
